<H1> <b>Capstone Project Data Science</b></H1> 

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [6]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

<h3>Get Data from wikipedia using BeautifulSoup</h3>

In [51]:
My_table = soup.find('table',{'class':'wikitable sortable'})
Columns_title=My_table.findAll("th")
i=0
df=pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        df.loc[len(df)+i]=[cells[0].find(text=True),cells[1].find(text=True),cells[2].find(text=True)]
        i=i+1 
        
           
            

<h3>Change type of DataFrame df to string</h3>

In [52]:
df=df.astype(str)
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

<h3> Select data where Borough different Not assigned </h3>

In [53]:
df=df.loc[df['Borough']!='Not assigned']

<h3>Replace all Neighbourhood=Not assigned by the value of Borough</h3>

In [54]:
df['Neighbourhood']=np.where(str(df['Neighbourhood'])=='Not assigned',df['Neighbourhood'] ,df['Borough'])
df.head(3)

,Postcode,Borough,Neighbourhood
4,M3A,North York,North York
6,M4A,North York,North York
8,M5A,Downtown Toronto,Downtown Toronto


<h3>Duplicate Postcode </h3>

In [55]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')
df.head(3)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Scarborough,Scarborough"
1,M1C,Scarborough,"Scarborough,Scarborough,Scarborough"
2,M1E,Scarborough,"Scarborough,Scarborough,Scarborough"


<h3>Shape of data</h3>

In [56]:
df.shape

(103, 3)

<h3>Get geographical coordinates  for each postcode</h3>

In [57]:
Geospatial=pd.read_csv('C:/Users/pc/Downloads/Geospatial_Coordinates.csv' ,error_bad_lines=False)

In [58]:
Geospatial.head()
Geospatial.rename(columns={'Postal Code':'Postcode' },inplace=True)
Geospatial.head(3)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [59]:
df=df.join(Geospatial.set_index('Postcode'), on='Postcode')
df.head(3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Scarborough,Scarborough",43.806686,-79.194353
1,M1C,Scarborough,"Scarborough,Scarborough,Scarborough",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Scarborough,Scarborough",43.763573,-79.188711
